# <u>Inventory Theory</u>

### Sets and Indices
- **$I$**: Set of products, indexed by $i$.
- **$T$**: Set of time periods, indexed by $t$.

### Parameters
- **$D_{i,t}$**: Demand for products $i$ in period $t$.
- **$C_i$**: Ordering cost per unit of product $i$.
- **$H_i$**: Holding cost per unit of product $i$ per period.
- **$B$**: Total budget for ordering products across all periods.
- **$S$**: Total storage capacity of the warehouse.
- **$V_i$**: Volume per unit of product $i$.

### Decision Variables
- **$x_{i,t}$**: Quantity of product $i$ to order in period $t$.
- **$y_{i,t}$**: Quantity of product $i$ held in inventory at the end of period $t$.

### Objective Function
$$
\text{Minimze} \quad Z = \sum_{i \in I} \sum_{t \in T} (C_i \cdot x_{i,t} + H_i \cdot y_{i,t})
$$
The objective function aims to minimze the total cost, which includes both ordering and holding costs for all products across all periods.

### Constraints

1. Inventory Balance:
$$
y_{i,t-1} + x_{i,t} = D_{i,t} + y_{i,t} \quad \forall i \in I , \forall t \in T
$$

2. Budget Constraint:
$$
\sum_{i \in I} C_i \cdot x_{i,t} \leq B \quad \forall t \in T
$$

3. Storage Capacity:
$$
\sum_{i \in I} V_i \cdot y_{i,t} \leq S \quad \forall t \in T
$$

4. Non-negativity:
$$
x_{i,t},y_{i,t} \geq 0 \quad \forall i \in I , \forall t \in T

---

In [16]:
from gurobipy import Model, GRB

In [17]:
# Create a new model
m = Model('inventory_optimization')

In [18]:
# Indices and Parameters
products = range(3) # Example: 3 products
periods = range(4) # Example: 4 periods

In [19]:
## Example data
D = {(0,0): 100, (0,1): 150, (0,2):200, (0,3):250, # Demand for product 0
     (1,0):80,   (1,1):120,  (1,2):160, (1,3):200, # Demand for product 1
     (2,0):50,   (2,1):75,   (2,2):100, (2,3):125} # Demand for product 2

In [20]:
C = {0:2, 1:3, 2:1.5}   # Ordering cost per product
H = {0:0.5, 1:0.7, 2:0.4}   # Holding cost per product
B = 1000 # Total budget
S = 500 # Total capacity
V = {0:1, 1:2, 2:1.5} # Volume per unit per product

In [21]:
# Decision Variables
x = m.addVars(products, periods, name='order', vtype=GRB.CONTINUOUS)
y = m.addVars(products, periods, name='hold', vtype=GRB.CONTINUOUS)

In [22]:
# Objective Function
m.setObjective(sum(C[i] * x[i,t] + H[i] * y[i,t] for i in products for t in periods), GRB.MINIMIZE)

In [23]:
# Constraints
# Inventory balance:
for i in products:
    for t in periods:
        m.addConstr(y[i,t-1] + x[i,t] == D[i,t] + y[i,t] if t > 0 else x[i,t] == D[i,t] + y[i,t])

# Budget constraint
for t in periods:
    m.addConstr(sum(C[i] * x[i,t] for i in products) <= B)

# Storage capacity constraint
for t in periods:
    m.addConstr(sum(V[i] * y[i,t] for i in products) <= S)

In [24]:
# Solving the model
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20 rows, 24 columns and 57 nonzeros
Model fingerprint: 0x8053092a
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [4e-01, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 1e+03]
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s
Presolved: 17 rows, 21 columns, 51 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.1500000e+02   2.700000e+02   0.000000e+00      0s
      16    3.7671667e+03   0.000000e+00   0.000000e+00      0s

Solved in 16 iterations and 0.04 seconds (0.00 work units)
Optimal objective  3.767166667e+03


In [25]:
# Output results
if m.Status == GRB.OPTIMAL:
    print(f'Optimal objective value: {m.objVal}')
    for v in m.getVars():
        print(f'{v.varName}: {v.x}')

Optimal objective value: 3767.1666666666674
order[0,0]: 100.0
order[0,1]: 150.0
order[0,2]: 200.0
order[0,3]: 250.0
order[1,0]: 110.00000000000006
order[1,1]: 195.83333333333334
order[1,2]: 150.0
order[1,3]: 104.16666666666667
order[2,0]: 50.0
order[2,1]: 75.0
order[2,2]: 100.0
order[2,3]: 125.0
hold[0,0]: 0.0
hold[0,1]: 0.0
hold[0,2]: 0.0
hold[0,3]: 0.0
hold[1,0]: 30.00000000000005
hold[1,1]: 105.83333333333336
hold[1,2]: 95.83333333333334
hold[1,3]: 0.0
hold[2,0]: 0.0
hold[2,1]: 0.0
hold[2,2]: 0.0
hold[2,3]: 0.0
